In [29]:
import arcpy
import os
import sys
ws = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb'
arcpy.env.workspace = ws
arcpy.env.overwriteOutput = True

In [3]:
def unique_values(table, field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
    
        return sorted({row[0] for row in cursor if row[0] != None})

In [4]:
def pubFind(searchTerms, myValues):
    pubLand = []
    myValues = [value.upper() for value in myValues]
    
    for owner in myValues:
        for term in searchTerms:
            if term in owner:
                pubLand.append(owner)
    return pubLand

In [5]:
def removeTerms(OgList, removeList):
    for item in removeList:
        if item in OgList:
            OgList.remove(item)
    return OgList

In [6]:
def removePart(OgList, strPrtList):
    newRemoveList = []
   
    for i in OgList:
        for string in strPrtList:
            if string in i:
                
                newRemoveList.append(i)
    
    
    for item in newRemoveList:
       
        if item in OgList:
            OgList.remove(item)
    
    return OgList, newRemoveList
    

In [7]:
def genRemoveList(OgList, strPrtList):
    newRemoveList = []
    
    for i in OgList:
        for string in strPrtList:
            if string in i:
                print(i)
                newRemoveList.append(i)
    return newRemoveList

In [8]:
def sqlClause(termList, fieldName):
    owner_string = ""
    for owner in termList:
        if termList.index(owner) != len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}' Or "
        elif termList.index(owner) == len(termList) -1:
            owner_string = owner_string + f"{fieldName} = '{owner}'"
    return owner_string

In [9]:
def narrowList(ogList):
    ogList = [i for i in ogList if not ('ESTATE' in i or 'SUSAN' in i or 'LLC' in i or 
                                                'COMPANY' in i or 'ASSOC' in i or 'ASSN' in i or
                                        'INC' in i or 'BANK' in i or 'CHURCH' in i or 'SOCIETY' in i or
                                               'RAILWAY' in i or 'PRESBYTERIAN' in i or 'CORP' in i or
                                       'FUND' in i)]
    
    removeList = [i for i in ogList if ('US' in i and not i.startswith('US'))]
    
    ogList = removeTerms(ogList, removeList)
    
    return ogList

In [10]:
def addAndFillField(fc, termList, newFcName, ownerFieldName):
    
    
    out_featureClass = os.path.join(ws, newFcName)
    arcpy.CopyFeatures_management(fc, out_featureClass)
    arcpy.AddField_management(out_featureClass, 'PublicLand', 'TEXT', 
                              field_length = 1)

    with arcpy.da.UpdateCursor(out_featureClass, 'PublicLand', sqlClause(termList, ownerFieldName)) as cursor:
        for row in cursor:
            row[0] = 'T'
            cursor.updateRow(row)
    

In [11]:
def addAndFillFieldKeithStyle(fc, termList, newFcName, ownerFieldName):
    
    
    out_featureClass = os.path.join(ws, newFcName)
    arcpy.CopyFeatures_management(fc, out_featureClass)
    arcpy.AddField_management(out_featureClass, 'PUB_PVT', 'TEXT', 
                              field_length = 7)

    with arcpy.da.UpdateCursor(out_featureClass, 'PUB_PVT', sqlClause(termList, ownerFieldName)) as cursor:
        for row in cursor:
            row[0] = 'PUBLIC'
            cursor.updateRow(row)

In [12]:
def addPrivateValues(fc):
    with arcpy.da.UpdateCursor(fc, 'PUB_PVT') as cursor:
        for row in cursor:
            if row[0] != 'PUBLIC':
                row[0] = 'PRIVATE'
            cursor.updateRow(row)

In [13]:
def addFalseValues(fc):
    with arcpy.da.UpdateCursor(fc, 'PublicLand') as cursor:
        for row in cursor:
            if row[0] != 'T':
                row[0] = 'F'
            cursor.updateRow(row)

In [14]:
%store -r pubAddList
%store -r refRemoveList

# Generate Owner Class Field for Pacific County

In [15]:
### Store all layers
clarkLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\2011\Clark_County_Parcels_clip_hgm.shp'
clatsLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClatsopCo\2010\Clatsop_taxlot_clip_hgm.shp'
colLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ColumbiaCo\2009\taxlot_09_study_area_with_ownership.shp'
cowLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\CowlitzCo\2005\aeparc_clip_hgm.shp'
multLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\MultCo\2011\multnomah_taxlots_clip_hgm_reach.shp'
pacLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\PacificCo\oldLayer\Tax_Lots.shp'
skaLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\SkamaniaCo\OldLayer\Skamania_Tax.shp'
wahLyr = r'C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\WahkiakumCo\OldLayer\Wahkiakum_Parcels_clip_hgm.shp'


In [16]:
### Pacific county has no owner_class so need to find public/private owners for pacific county
myValues = unique_values(pacLyr, 'CURRENT_O')
searchTerms = ['UNITED', 'STATE', 'US ', 'USA ', 'OREGON', 'WASH', 
            'COUNTY', 'PACIFIC', 'CITY', 'GOVERNMENT', 'SCHOOL', 'PUBLIC']
pacPubOwners = pubFind(searchTerms, myValues)
pacPubOwners = narrowList(pacPubOwners)
removeUs = [i for i in pacPubOwners if ('US' in i and not i.startswith('US'))]
pacPubOwners = removeTerms(pacPubOwners, removeUs)
removeList = ['ANNIE WRIGHT SCHOOL,', 'DURYEE & SAUNDERS PACIFIC COUNTY REAL*', 'DURYEE & SAUNDERS PACIFIC COUNTY REAL*', 
              'EAST OREGONIAN PUBLISHING CO,', 'FIA TIMBER GROWTH & VALUE WASHINGTON *', 'FRIENDS OF CHINOOK SCHOOL,', 
              'LEWIS COUNTY COMMUNITY HEALTH SERVICE*', 'PACIFIC COAST CRANBERRY RESEARCH,', 'PACIFIC COUNTY ANGLERS,', 
              'PACIFIC COUNTY ANGLERS,', 'PACIFIC EYE CLINIC,', 'PACIFIC NW ANNUAL CONF/UNITED METHODI*', 
              'PACIFIC NW ANNUAL CONF/UNITED METHODI*', 'PACIFIC PLACE APT LMT,',"RAYMOND WASH CONG OF JEHOVAH'S WITNES*", 
             'SCHOOLING, JACK E & DONELLE J', 'STATEN, JACQUALINE M', 'SUNRISE WASHINGTON SUBSIDIARY CO,',
             'WASHAM, DANIEL & HEIDI & GLENN', 'WASHAM, GLENN', 'WASHINGTON CONFERENCE OF SEVENTH DAY *', 
              'WESTERN WASH CONS BAPTIST CAMPING ASS*', 'WESTERN WASH CONSERV BAPTIST,', 'WESTERN WASH CORP 7TH DAY ADVEN,'] + refRemoveList
pacPubOwners = removeTerms(pacPubOwners, removeList)
pacPubOwners = pacPubOwners + pubAddList

In [17]:
discRemove = ['PORT OF', 'SCHOOL', 'EDUC', 'UNIVER', 'COLLEGE', 'LEAS', 'PUD',
             'UTILITY', 'UTILITIES', 'UTITLITIES', 'UTIL DIST','PUBLIC WORKS', 'WATER WORKS', 'WATER BUREAU', 
              'CEMETERY', 'HOSPITAL', 'FIRE AND RESCUE', 'FIRE & RESCUE', 'LIBRAR',
             'DRAIN', 'HIGHER ED', 'HIGHWAY', 'HWY', 'FIRE/RESCUE',
             'FIRE DIST', 'PORT DISTRICT', 'BOYS & GIRLS', 'EVA MARIA DEMETRO',
             'ACADEMY', 'HEALTHCARE', 'FIRE PROTECTION']
pacPubOwners, pacRemove = removePart(pacPubOwners, discRemove)

In [18]:
pacPubOwners = removeTerms(pacPubOwners, pacRemove)

In [19]:
cityRemove = [i for i in pacPubOwners if ('CITY' in i)]
cityRemove = [i for i in cityRemove if not ('PARK' in i or 'METRO' in i or 'ENVIRONMENTAL' in i or 'WATER RES' in i or 
                                            'HOUSING AUTHORITY' in i or 'WILDLIFE' in i)]

In [20]:
pacPubOwners = removeTerms(pacPubOwners, cityRemove)

In [21]:
newRemoveList = ['EAST OREGONIAN PUBLISHING CO', 'P U D #1 GRAYS HARBOR COUNTY', 
                 'PACIFIC COAST CRANBERRY RESEARCH', 'PACIFIC EYE CLINIC', 'PACIFIC FOREST MANAGEMENT & HARVEST LL', 'PACIFIC NW ANNUAL CONF/UNITED METHODIS', 'PACIFIC NW ANNUAL CONF/UNITED METHODIS', 'PACIFIC NW DIST', 'PACIFIC PLACE APT LMT', 'PACIFIC POST #968 VFW', 'PACIFIC SOLID WASTE DISPOSAL', 
                 'PACIFIC TRANSIT SYSTEM', 'PUBLIC HEALTH CARE SERVICES #3 OF PACI',
                'STATEN, JAY P & VALERIE A', 'SUNRISE WASHINGTON SUBSIDIARY CO', 'THE ARC OF CLARK COUNTY',
                'WASHAM, JAMES & ROBIN', 'WESTERN WASH CONSERV BAPTIST']
pacPubOwners = removeTerms(pacPubOwners, newRemoveList)

In [22]:
addAndFillFieldKeithStyle(pacLyr, pacPubOwners, 'oldPacOwners', 'CURRENT_O')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\TaxLots\TaxLots.gdb\oldPacOwners already exists.
Failed to execute (CopyFeatures).


In [23]:
addPrivateValues('oldPacOwners')

# Merge the County Layers Together

In [25]:

pacLyr = 'oldPacOwners'
coList = [clarkLyr, clatsLyr, colLyr, cowLyr, multLyr, pacLyr, skaLyr, wahLyr]
clarkOwn, clatsOwn, colOwn, cowOwn, multOwn, pacOwn, skaOwn, wahOwn = 'OWNER', 'OWNER_LINE', 'OWNER', 'DEED_HOLDE', 'OWNER1', 'CURRENT_O', 'CURNT_OWN', 'Own_Last'
ownList = [clarkOwn, clatsOwn, colOwn, cowOwn, multOwn, pacOwn, skaOwn, wahOwn]
clarkPubField, clatsOwn, colPubField, coPubField, multPubFiel, pacPubField, skaPubField, wahPubField = 'PUB_PVT', 'pub_pvt', 'PUB_PVT', 'PUB_PVT', 'pub_pvt', 'PUB_PVT', 'ownClass', 'PUB_PVT'
pubPvtList = [clarkPubField, clatsOwn, colPubField, coPubField, multPubFiel, pacPubField, skaPubField, wahPubField]

In [27]:
#make sure all in the same projection
for co in coList:
    spatial_ref = arcpy.Describe(co).spatialReference
    print(f'{co}:{spatial_ref.name}, {spatial_ref.linearUnitName}')

C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClarkCo\2011\Clark_County_Parcels_clip_hgm.shp:NAD_1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet, Foot_US
C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ClatsopCo\2010\Clatsop_taxlot_clip_hgm.shp:NAD_1983_HARN_StatePlane_Oregon_North_FIPS_3601_IntlFeet, Foot
C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\ColumbiaCo\2009\taxlot_09_study_area_with_ownership.shp:NAD_1983_HARN_StatePlane_Oregon_North_FIPS_3601_Feet_Intl, Foot
C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\CowlitzCo\2005\aeparc_clip_hgm.shp:NAD_1983_StatePlane_Washington_South_FIPS_4602_Feet, Foot_US
C:\Users\Ricardo\Desktop\PCC\Internship\UpdatedCountyTaxLots\Shapefiles\MultCo\2011\multnomah_taxlots_clip_hgm_reach.shp:NAD_1983_HARN_StatePlane_Oregon_North_FIPS_3601, Foot
oldPacOwners:NAD_1983_StatePlane_Washington_South_FIPS_4602_Feet, Foot_US
C:\Users\Ricardo\Desktop\PCC

In [28]:
# Project skamania to project in feet
# Reproject multnomah county to appropriate projection
arcpy.Project_management(skaLyr, 'oldSkamaniaProject', arcpy.Describe(wahLyr).spatialReference)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\oldSkamaniaProject'>

In [30]:
pubPvtFldMap = arcpy.FieldMap()
landOwnerFldMap = arcpy.FieldMap()


fms = arcpy.FieldMappings()

In [31]:
for co, own, pubPvt in zip(coList, ownList, pubPvtList):
    pubPvtFldMap.addInputField(co, pubPvt)
    landOwnerFldMap.addInputField(co, own)

In [32]:
pubPvt_name = pubPvtFldMap.outputField
pubPvt_name.name = 'PUB_PVT'
pubPvt_name.length = 100
pubPvtFldMap.outputField = pubPvt_name

landOwner_name = landOwnerFldMap.outputField
landOwner_name.name = 'Land Owner'
landOwner_name.length = 100
landOwnerFldMap.outputField = landOwner_name


In [33]:
fms.addFieldMap(landOwnerFldMap)
fms.addFieldMap(pubPvtFldMap)

In [34]:
arcpy.Merge_management(coList, 'old_parcels_merge', fms)

<Result 'C:\\Users\\Ricardo\\Desktop\\PCC\\Internship\\UpdatedCountyTaxLots\\TaxLots\\TaxLots.gdb\\old_parcels_merge'>

# Capitalize all field entries in PUB_PVT field 

In [35]:
with arcpy.da.UpdateCursor('old_parcels_merge', 'PUB_PVT') as cursor:
    for row in cursor:
        row[0] = row[0].upper()
        if row[0].upper() != 'PRIVATE' and row[0].upper() != 'UNSURE':
            row[0] = 'PUBLIC'
            
        cursor.updateRow(row)
del cursor, row